# 📈 Analyse Long-terme des Recettes

## Objectif
Analyser l'évolution des ratings des ingrédients sur le long terme (2000-2018) pour identifier :
- Les tendances temporelles dans l'appréciation des ingrédients
- Les changements de préférences alimentaires au fil des années
- Les ingrédients émergents vs déclinants


In [9]:
# Import des bibliothèques nécessaires
import sys
sys.path.append('..')

# 🎯 Chargement des fonctions utilitaires depuis le fichier Python (plus robuste)
from _data_utils import *

# 📦 Imports et chargement des données
import sys
from pathlib import Path
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


# 🎨 Configuration des graphiques
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Imports réussis!")

✅ Imports réussis!


## 📊 Chargement et nettoyage des données

Utilisation de `load_clean_recipes()` qui :
- Charge la table RAW_recipes depuis DuckDB
- Nettoie les données (supprime nulls, doublons)
- Parse la colonne nutrition en 7 colonnes séparées
- Ajoute les features temporelles (year, month, weekday, season, is_weekend)
- Calcule le score de complexité

In [10]:
# 📥 Chargement des recettes avec nettoyage automatique
df = load_clean_recipes()

print(f"✅ Recettes chargées : {df.shape[0]:,} lignes, {df.shape[1]} colonnes")
print(f"📅 Période couverte : {df['year'].min()} - {df['year'].max()}")
print(f"\n📋 Colonnes disponibles :")
print(df.columns)

✅ RAW_recipes chargée (231637 lignes, 12 colonnes)


SchemaError: invalid series dtype: expected `String`, got `date` for series with name `submitted`

## 🔍 Analyse de la qualité des données

Vérification rapide de la qualité des données chargées.

In [ ]:
# 🔬 Rapport de qualité automatique
quality_report = analyze_recipe_quality(df)

# Afficher un aperçu des données
print(f"\n📊 Aperçu des données :")
df.head(3)

## 📈 Analyse 1 : Volume de recettes dans le temps

Analyser l'évolution du nombre de recettes soumises par année, mois, weekend/weekday, et saison.

In [ ]:
# 📊 Agrégation des données par dimensions temporelles
by_year = df.group_by("year").len().sort("year").to_pandas()
by_month = df.group_by("month").len().sort("month").to_pandas()
by_weekend = df.group_by("is_weekend").len().sort("is_weekend").to_pandas()
by_season = df.group_by("season").len().sort("season").to_pandas()

# 🎨 Visualisation 4 graphiques
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Volume par année
sns.barplot(data=by_year, x="year", y="len", color="green", ax=axes[0,0])
axes[0,0].set_title("📅 Nombre de recettes par année", fontsize=12, fontweight='bold')
axes[0,0].set_xlabel("Année")
axes[0,0].set_ylabel("Nombre de recettes")
axes[0,0].tick_params(axis='x', rotation=45, labelsize=8)

# 2. Volume par mois
sns.barplot(data=by_month, x="month", y="len", ax=axes[0,1])
axes[0,1].set_title("📆 Nombre de recettes par mois", fontsize=12, fontweight='bold')
axes[0,1].set_xlabel("Mois")
axes[0,1].set_ylabel("Nombre de recettes")

# 3. Weekend vs Weekday
by_weekend['label'] = by_weekend['is_weekend'].map({0: 'Weekday', 1: 'Weekend'})
sns.barplot(data=by_weekend, x="label", y="len", palette=["skyblue", "orange"], ax=axes[1,0])
axes[1,0].set_title("📌 Weekday vs Weekend", fontsize=12, fontweight='bold')
axes[1,0].set_xlabel("")
axes[1,0].set_ylabel("Nombre de recettes")

# 4. Volume par saison
sns.barplot(data=by_season, x="season", y="len", palette="Set2", ax=axes[1,1])
axes[1,1].set_title("🌸 Nombre de recettes par saison", fontsize=12, fontweight='bold')
axes[1,1].set_xlabel("Saison")
axes[1,1].set_ylabel("Nombre de recettes")

plt.tight_layout()
plt.show()

print(f"📊 Résumé : {df.shape[0]:,} recettes entre {df['year'].min()} et {df['year'].max()}")

## ⏱️ Analyse 2 : Temps de préparation moyen (minutes)

Analyser comment le temps moyen de préparation des recettes évolue dans le temps.

**Note** : La fonction `load_clean_recipes()` filtre automatiquement :
- Les recettes avec `minutes <= 0`
- Les recettes avec `minutes > 180` (plus de 3 heures)

In [ ]:
# 📊 Calcul des temps moyens par dimension temporelle
avg_year = df.group_by("year").agg(pl.col("minutes").mean().alias("avg_minutes")).sort("year").to_pandas()
avg_month = df.group_by("month").agg(pl.col("minutes").mean().alias("avg_minutes")).sort("month").to_pandas()
avg_weekend = df.group_by("is_weekend").agg(pl.col("minutes").mean().alias("avg_minutes")).sort("is_weekend").to_pandas()
avg_season = df.group_by("season").agg(pl.col("minutes").mean().alias("avg_minutes")).sort("season").to_pandas()

# 🎨 Visualisation 4 graphiques
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Temps moyen par année
sns.barplot(data=avg_year, x="year", y="avg_minutes", color="skyblue", ax=axes[0,0])
axes[0,0].set_title("⏱️ Temps moyen par année", fontsize=12, fontweight='bold')
axes[0,0].set_xlabel("Année")
axes[0,0].set_ylabel("Temps moyen (minutes)")
axes[0,0].tick_params(axis='x', rotation=45, labelsize=8)

# 2. Temps moyen par mois
sns.barplot(data=avg_month, x="month", y="avg_minutes", color="orange", ax=axes[0,1])
axes[0,1].set_title("⏱️ Temps moyen par mois", fontsize=12, fontweight='bold')
axes[0,1].set_xlabel("Mois")
axes[0,1].set_ylabel("Temps moyen (minutes)")

# 3. Weekday vs Weekend
avg_weekend['label'] = avg_weekend['is_weekend'].map({0: 'Weekday', 1: 'Weekend'})
sns.barplot(data=avg_weekend, x="label", y="avg_minutes", palette=["lightgreen", "coral"], ax=axes[1,0])
axes[1,0].set_title("⏱️ Temps moyen Weekday vs Weekend", fontsize=12, fontweight='bold')
axes[1,0].set_xlabel("")
axes[1,0].set_ylabel("Temps moyen (minutes)")

# 4. Temps moyen par saison
sns.barplot(data=avg_season, x="season", y="avg_minutes", palette="viridis", ax=axes[1,1])
axes[1,1].set_title("⏱️ Temps moyen par saison", fontsize=12, fontweight='bold')
axes[1,1].set_xlabel("Saison")
axes[1,1].set_ylabel("Temps moyen (minutes)")

plt.tight_layout()
plt.show()

print(f"⏱️ Temps moyen global : {df['minutes'].mean():.1f} minutes")
print(f"📊 Médiane : {df['minutes'].median():.1f} minutes")

## 🔥 Analyse 3 : Évolution des calories moyennes

Analyser l'évolution nutritionnelle des recettes dans le temps.

**Note** : Les colonnes nutritionnelles ont été automatiquement parsées par `load_clean_recipes()` :
- `calories`, `total_fat_pct`, `sugar_pct`, `sodium_pct`, `protein_pct`, `sat_fat_pct`, `carb_pct`

In [ ]:
# 📊 Calories moyennes par année
calories_year = df.group_by("year").agg(pl.col("calories").mean().alias("avg_calories")).sort("year").to_pandas()

# 🎨 Visualisation avec ligne de tendance
fig, ax = plt.subplots(figsize=(14, 6))

# Barres
sns.barplot(data=calories_year, x="year", y="avg_calories", color="tomato", ax=ax)

# Ligne de tendance
ax.plot(calories_year.index, calories_year['avg_calories'], color='darkred', linewidth=2, marker='o', label='Tendance')

ax.set_title("🔥 Évolution des calories moyennes par année", fontsize=14, fontweight='bold')
ax.set_xlabel("Année", fontsize=12)
ax.set_ylabel("Calories moyennes", fontsize=12)
ax.tick_params(axis='x', rotation=45, labelsize=9)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 📊 Statistiques
print(f"🔥 Calories en {calories_year.iloc[0]['year']:.0f} : {calories_year.iloc[0]['avg_calories']:.1f} kcal")
print(f"🔥 Calories en {calories_year.iloc[-1]['year']:.0f} : {calories_year.iloc[-1]['avg_calories']:.1f} kcal")
variation = ((calories_year.iloc[-1]['avg_calories'] - calories_year.iloc[0]['avg_calories']) / calories_year.iloc[0]['avg_calories'] * 100)
print(f"📈 Variation : {variation:+.1f}%")

## 🎯 Analyse 4 : Score de complexité des recettes

Le score de complexité est calculé automatiquement par `compute_recipe_complexity()` :

$$\text{complexity\_score} = \frac{\text{minutes}}{10} + \text{n\_steps} + \text{n\_ingredients} \times 0.5$$

Plus le score est élevé, plus la recette est complexe.

In [ ]:
# 📊 Complexité moyenne par année
complexity_year = df.group_by("year").agg(
    pl.col("complexity_score").mean().alias("avg_complexity")
).sort("year").to_pandas()

# 🎨 Visualisation
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(complexity_year['year'], complexity_year['avg_complexity'], 
        marker='o', linewidth=2.5, color='purple', markersize=8)
ax.fill_between(complexity_year['year'], complexity_year['avg_complexity'], alpha=0.3, color='purple')

ax.set_title("🎯 Évolution du score de complexité moyen", fontsize=14, fontweight='bold')
ax.set_xlabel("Année", fontsize=12)
ax.set_ylabel("Score de complexité moyen", fontsize=12)
ax.grid(True, alpha=0.4)
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# 📊 Statistiques descriptives
print(f"📊 Score de complexité :")
print(f"   Minimum : {df['complexity_score'].min():.2f}")
print(f"   Maximum : {df['complexity_score'].max():.2f}")
print(f"   Moyenne : {df['complexity_score'].mean():.2f}")
print(f"   Médiane : {df['complexity_score'].median():.2f}")

## 📊 Analyse 5 : Distribution du nombre d'ingrédients

Explorer la distribution du nombre d'ingrédients par recette.

In [ ]:
# 📊 Distribution + évolution temporelle
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# 1. Histogramme de distribution
df['n_ingredients'].to_pandas().hist(bins=30, edgecolor='black', color='steelblue', ax=ax1)
ax1.axvline(df['n_ingredients'].median(), color='red', linestyle='--', linewidth=2, label=f"Médiane: {df['n_ingredients'].median()}")
ax1.axvline(df['n_ingredients'].mean(), color='orange', linestyle='--', linewidth=2, label=f"Moyenne: {df['n_ingredients'].mean():.1f}")
ax1.set_title("📊 Distribution du nombre d'ingrédients", fontsize=12, fontweight='bold')
ax1.set_xlabel("Nombre d'ingrédients")
ax1.set_ylabel("Fréquence")
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Évolution du nombre moyen d'ingrédients par année
ingredients_year = df.group_by("year").agg(
    pl.col("n_ingredients").mean().alias("avg_ingredients")
).sort("year").to_pandas()

ax2.plot(ingredients_year['year'], ingredients_year['avg_ingredients'], 
         marker='s', linewidth=2, color='forestgreen', markersize=7)
ax2.set_title("📈 Évolution du nombre moyen d'ingrédients par année", fontsize=12, fontweight='bold')
ax2.set_xlabel("Année")
ax2.set_ylabel("Nombre moyen d'ingrédients")
ax2.grid(True, alpha=0.3)
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"🥕 Nombre d'ingrédients :")
print(f"   Médiane : {df['n_ingredients'].median()}")
print(f"   Moyenne : {df['n_ingredients'].mean():.1f}")
print(f"   Min : {df['n_ingredients'].min()}")
print(f"   Max : {df['n_ingredients'].max()}")

## 🎁 Résumé des insights

### 📌 Principales observations

Synthèse des tendances observées dans l'analyse long-terme des recettes.

In [ ]:
# 📝 Génération automatique du résumé
print("=" * 70)
print("📊 RAPPORT D'ANALYSE - RECETTES (2000-2018)")
print("=" * 70)
print(f"\n📦 Volume de données :")
print(f"   • Total recettes analysées : {df.shape[0]:,}")
print(f"   • Période : {df['year'].min()} - {df['year'].max()}")
print(f"   • Colonnes disponibles : {df.shape[1]}")

print(f"\n⏱️  Temps de préparation :")
print(f"   • Moyenne : {df['minutes'].mean():.1f} minutes")
print(f"   • Médiane : {df['minutes'].median():.1f} minutes")

print(f"\n🔥 Nutrition :")
print(f"   • Calories moyennes : {df['calories'].mean():.1f} kcal")
print(f"   • Variation 2000→2018 : {((calories_year.iloc[-1]['avg_calories'] - calories_year.iloc[0]['avg_calories']) / calories_year.iloc[0]['avg_calories'] * 100):+.1f}%")

print(f"\n🎯 Complexité :")
print(f"   • Score moyen : {df['complexity_score'].mean():.2f}")
print(f"   • Médiane : {df['complexity_score'].median():.2f}")

print(f"\n🥕 Ingrédients :")
print(f"   • Nombre moyen : {df['n_ingredients'].mean():.1f}")
print(f"   • Médiane : {df['n_ingredients'].median()}")

print(f"\n📅 Saisonnalité :")
season_counts = df.group_by("season").len().sort("len", descending=True).to_pandas()
for idx, row in season_counts.iterrows():
    pct = (row['len'] / df.shape[0] * 100)
    print(f"   • {row['season']}: {row['len']:,} recettes ({pct:.1f}%)")

print(f"\n🔄 Weekend vs Weekday :")
weekend_counts = df.group_by("is_weekend").len().to_pandas()
for idx, row in weekend_counts.iterrows():
    label = "Weekend" if row['is_weekend'] == 1 else "Weekday"
    pct = (row['len'] / df.shape[0] * 100)
    print(f"   • {label}: {row['len']:,} recettes ({pct:.1f}%)")

print("\n" + "=" * 70)
print("✅ Analyse terminée avec succès !")
print("=" * 70)